In [ ]:
pip install tensorflow-gpu==2.5

In [ ]:
import tensorflow as tf

In [ ]:
pip install imblearn

In [ ]:
pip install opencv

In [ ]:
import cv2 
import tensorflow as tf 
import numpy as np 
from sklearn import metrics
from sklearn.metrics import confusion_matrix,f1_score
# import matplotlib.pyplot as plt
from tqdm import tqdm
import time
import tensorflow.keras
from imblearn.over_sampling import SMOTE
from tensorflow.keras.layers import Conv2D, BatchNormalization,MaxPool2D, GlobalMaxPool2D,LSTM,Bidirectional
from tensorflow.keras.layers import TimeDistributed, GRU, Dense, Dropout
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder

In [ ]:
def build_convnet(shape=(112, 112, 3)):
    momentum = .9
    model = tensorflow.keras.Sequential()
    model.add(Conv2D(64, (3,3), input_shape=shape,
    padding='same', activation='relu'))
    model.add(Conv2D(64, (3,3), padding='same', activation='relu'))
    model.add(BatchNormalization(momentum=momentum))
    model.add(MaxPool2D())
    model.add(Conv2D(128, (3,3), padding='same', activation='relu'))
    model.add(Conv2D(128, (3,3), padding='same', activation='relu'))
    model.add(BatchNormalization(momentum=momentum))
    model.add(MaxPool2D())
    model.add(Conv2D(256, (3,3), padding='same', activation='relu'))
    model.add(Conv2D(256, (3,3), padding='same', activation='relu'))
    model.add(BatchNormalization(momentum=momentum))
    model.add(MaxPool2D())
    model.add(Conv2D(512, (3,3), padding='same', activation='relu'))
    model.add(Conv2D(512, (3,3), padding='same', activation='relu'))
    model.add(BatchNormalization(momentum=momentum))
    model.add(GlobalMaxPool2D())
    return model

In [ ]:
m=build_convnet(shape=(112, 112, 3))

In [ ]:
def action_model(shape=(10, 112, 112, 3), nbout=2):
    convnet = build_convnet(shape[1:])
    model = tensorflow.keras.Sequential()
    model.add(TimeDistributed(convnet, input_shape=shape))
    model.add(Bidirectional(LSTM(32, return_sequences=True)))
    model.add(Bidirectional(LSTM(64, return_sequences=True)))
    model.add(LSTM(64))
    model.add(Dense(1024, activation='relu'))
    model.add(Dropout(.5))
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(.5))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(.5))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(nbout, activation='softmax'))
    return model

In [ ]:
NBFRAME=10
SIZE=(112,112)
CHANNELS=3

In [ ]:
INSHAPE=(NBFRAME,) + SIZE + (CHANNELS,) # (5, 112, 112, 3)
print(INSHAPE)
model = action_model((10, 112, 112, 3),2)
optimizer = tensorflow.keras.optimizers.Adam(0.001)
model.compile(optimizer,'sparse_categorical_crossentropy',metrics=['acc'])

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import Sequential, load_model

In [ ]:
from imblearn.under_sampling import RandomUnderSampler

In [ ]:
#for fakeavceleb dataset
Real_features= np.load("F:\\fakeavceleb_balance\\data npy format\\bi-lstm\\1651_real video real audio_train.npy",allow_pickle=True)

In [ ]:
#for fakeavceleb dataset
Fake_features=np.load("F:\\fakeavceleb_balance\\data npy format\\bi-lstm\\6187_fakevideo fake audio_train.npy",allow_pickle=True)

In [ ]:
IMAGE=[]
for i in tqdm(range(Total_features.shape[0])):
    for j in range(1,np.array(Total_features[i]).shape[0]):
        im=Total_features[i][j]
        IMAGE.append(im)

In [ ]:
IMAGE=np.array(IMAGE)

In [ ]:
IMAGE=IMAGE.reshape(-1,112,112,3)

In [ ]:
label1=[]
for i in tqdm(range(Total_label.shape[0])):
    for j in range(1,np.array(Total_label[i]).shape[0]):
        im1=Total_label[i][j]
        label1.append(im1)

In [ ]:
label=[]
for i in range(len(label1)):
  if label1[i]==0:
    label.append([1,0])
  else:
    label.append([0,1])

In [ ]:
label=np.array(label)

In [ ]:
Total_features=np.load('F:\\fakeavceleb_balance\\data npy format\\bi-lstm\\ALL_TRAIN.npy')

In [ ]:
Total_label=np.load('F:\\fakeavceleb_balance\\data npy format\\bi-lstm\\ALL_LABEL.npy')

In [ ]:
X_train,X_test,Y_train,Y_test=train_test_split(np.array(Total_features),np.array(Total_label),random_state=42,test_size=0.2)

In [ ]:
model.fit(X_train, Y_train,batch_size=64,epochs=40,verbose=1,validation_data=(X_test,Y_test))

In [ ]:
results = model.evaluate(X_test, Y_test)

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix,classification_report,roc_curve, auc
from sklearn.metrics import precision_score, recall_score, f1_score

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
threshold = 0.5
y_pred_binary = (y_pred > threshold).astype(int)

In [ ]:
pred=[]
for i in range(len(y_pred_binary)):
    pred.append(np.argmax(y_pred_binary[i]))

In [ ]:
pred=np.array(pred)

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

# Assuming y_true and y_pred_binary are your true labels and predicted labels
# y_true should be in multilabel-indicator format, and y_pred_binary should be the predicted binary labels

# Choose an appropriate averaging strategy
# Example: micro, macro, weighted, or None
average_strategy = 'macro'

# Calculate precision, recall, and f1 score with the chosen averaging strategy
precision = precision_score(Y_test, pred,average=average_strategy)
recall = recall_score(Y_test,pred, average=average_strategy)
f1 = f1_score(Y_test, pred, average=average_strategy)

print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

In [ ]:
print(classification_report(Y_test,pred))

In [ ]:
from sklearn.metrics import confusion_matrix
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
y_test_1d=Y_test.ravel()
y_pred_binary_1d=pred.ravel()

In [ ]:
cm =confusion_matrix(y_test_1d, y_pred_binary_1d)

# Print confusion matrix
print("Confusion Matrix:")
print(cm)

# Plot confusion matrix using seaborn and matplotlib
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", cbar=False,
            xticklabels=np.unique(y_test_1d), yticklabels=np.unique(y_test_1d))
plt.title("Confusion Matrix")
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Plot training & validation accuracy values
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

# Plot training & validation loss values
# plt.subplot(1, 2, 2)
# plt.plot(history.history['loss'])
# plt.plot(history.history['val_loss'])
# plt.title('Model loss')
# plt.ylabel('Loss')
# plt.xlabel('Epoch')
# plt.legend(['Train', 'Validation'], loc='upper left')

# plt.show()

In [ ]:
# y_test = np.array([0, 0, 1, 1, 0, 1, 0, 1, 0, 1])  # True labels
# y_pred_prob = np.array([0.1, 0.4, 0.35, 0.8, 0.2, 0.65, 0.5, 0.9, 0.45, 0.85])  # Predicted probabilities

# Calculate False Positive Rate, True Positive Rate, and thresholds
fpr, tpr, thresholds = roc_curve(Y_test, y_pred_binary_1d)

# Calculate the Area Under the Curve (AUC)
roc_auc = auc(fpr, tpr)

# Plot ROC curve
plt.figure()
plt.plot(fpr, tpr, color='blue', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='gray', lw=2, linestyle='--')  # Random guess line
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate (FPR)')
plt.ylabel('True Positive Rate (TPR)')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc="lower right")
plt.show()